In [161]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer

# Dataset 불러오기 60bpm 이상

In [162]:
df = pd.read_csv('tempo_upper_60.csv')
df['genre_list'] = df['track_genre'].apply(lambda x: [g.strip() for g in x.split(',')])

# 2. MultiLabelBinarizer로 인코딩
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre_list'])

# 3. 결과 DataFrame에 붙이기
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
df = pd.concat([df, genre_df], axis=1)

# ISO 원리를 위한 tempo별 분리

In [163]:
def classify_iso_tempo(tempo): 
    if tempo < 70:
        return 'Stage 1'
    elif tempo < 80:
        return 'Stage 2'
    elif tempo < 90:
        return 'Stage 3'
    elif tempo < 100:
        return 'Stage 4'
    elif tempo < 110:
        return 'Stage 5'
    elif tempo < 120:
        return 'Stage 6'
    else:
        return 'Stage 7'

In [164]:
# 분류 적용
df['iso_stage'] = df['tempo'].apply(classify_iso_tempo)

In [165]:
df.head()

,track_id,track_name,artist_name,year,popularity,artwork_url,album_name,acousticness,danceability,duration_ms,...,urbano latino,vallenato,vaporwave,variété française,visual kei,vocal jazz,vocaloid,west coast hip hop,worship,iso_stage
0,0BBAcsIw7RYLtHFTNJUQFe,Knockout Song - Ambient Lofi,"Anirudh Ravichander, Madley Blues, Vignesh Shi...",2024,12,https://i.scdn.co/image/ab67616d0000b2738aa5c0...,Knockout Song (Ambient Lofi),0.4510,0.609,98400,...,0,0,0,0,0,0,0,0,0,Stage 4
1,5OMH9KGnXhzbiVL1kDxZTg,Knockout Song - House Mix,"Anirudh Ravichander, Madley Blues, Vignesh Shi...",2023,7,https://i.scdn.co/image/ab67616d0000b27361845d...,Knockout Song (House Mix),0.0264,0.753,84375,...,0,0,0,0,0,0,0,0,0,Stage 7
2,3sOQaED4oONy3cMR8ywwE5,Vaaran Unna Vechiseyya Poraan,"Santhosh Hariharan, Shenbagaraj, Vignesh Naray...",2019,0,https://i.scdn.co/image/ab67616d0000b2730eb73d...,Kennedy Club (Original Motion Picture Soundtrack),0.4160,0.690,115882,...,0,0,0,0,0,0,0,0,0,Stage 7
3,5zDRx2IlpoiNHBpmG4btVN,"Per Vachaalum Vaikkaama (From ""Dikkiloona"")","Yuvan Shankar Raja, Ilaiyaraaja, Malaysia Vasu...",2021,51,https://i.scdn.co/image/ab67616d0000b27377e32a...,"Per Vachaalum Vaikkaama (From ""Dikkiloona"")",0.3050,0.761,268400,...,0,0,0,0,0,0,0,0,0,Stage 2
4,21r0QnhYxsxYUsoov5o0IF,Per Vachaalum Vaikkaama,"Yuvan Shankar Raja, Ilaiyaraaja, Malaysia Vasu...",2021,47,https://i.scdn.co/image/ab67616d0000b2736dd803...,Dikkiloona (Original Motion Picture Soundtrack),0.3020,0.754,268400,...,0,0,0,0,0,0,0,0,0,Stage 2


# 취향 반영을 위한 사용자 선택 음악 입력

In [166]:
features = ['popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence'] + list(mlb.classes_)

In [175]:
current_iso_stage = df[df['iso_stage']=='Stage 1'].copy() # BPM 대역 선택
dynamite_features=[62, 0.0112, 0.746, 0.765, 0.0, 6, 0.0933, -4.41, 0, 0.0993, 114.044, 0.737]  # 사용자 취향의 음악
dynamite_genre = ['k-pop'] # 사용자 취향 장르

dynamite_genre_encoded = mlb.transform([dynamite_genre])[0]
dynamite_all_features = np.array(dynamite_features + dynamite_genre_encoded.tolist())

# input 전처리
dynamite_df = pd.DataFrame([dynamite_all_features], columns=features)

In [176]:
girlsLikeYou_feature = [28,0.558,0.604,0.462,1.05E-6,0,0.131,-7.053,1,0.0614,60.26,0.378]
girlsLikeYou_genre = ['pop']

girlsLikeYou_genre_encoded = mlb.transform([girlsLikeYou_genre])[0]
girlsLikeYou_all_feature = np.array(girlsLikeYou_feature + girlsLikeYou_genre_encoded.tolist())

girlsLikeYou_df = pd.DataFrame([girlsLikeYou_all_feature],columns=features)

# 스케일링

In [177]:
scaler = MinMaxScaler()
current_scaled = scaler.fit_transform(current_iso_stage[features])
dynamite_scaled = scaler.transform(dynamite_df)

In [178]:
girlsLikeYou_scaled = scaler.transform(girlsLikeYou_df)

# 코사인 유사도 계산

In [179]:
similarities = cosine_similarity(dynamite_scaled, current_scaled)[0]

# 4. 유사도 추가 후 상위 N개 추천
current_iso_stage['similarity'] = similarities
top_recommendations = current_iso_stage.sort_values(by='similarity', ascending=False).head(10)

In [180]:
similarities = cosine_similarity(girlsLikeYou_scaled, current_scaled)[0]

current_iso_stage['similarity_2'] = similarities
top_recommendations_2 = current_iso_stage.sort_values(by='similarity_2', ascending=False).head(10)

In [181]:
top_recommendations

,track_id,track_name,artist_name,year,popularity,artwork_url,album_name,acousticness,danceability,duration_ms,...,vallenato,vaporwave,variété française,visual kei,vocal jazz,vocaloid,west coast hip hop,worship,iso_stage,similarity
4510,4FopzmRUfn8Ob8xlYVZqe8,Close to You (SEUNGMIN of Stray Kids),Stray Kids,2022,27,https://i.scdn.co/image/ab67616d0000b273abfe8d...,Love In Contract (Original Television Soundtrack),0.71100,0.516,228539,...,0,0,0,0,0,0,0,0,Stage 1,0.742932
2423,58ItBuVuKc03DvqSeZFnH1,Sacrifice (Eat Me Up),ENHYPEN,2023,70,https://i.scdn.co/image/ab67616d0000b2731d03b5...,DARK BLOOD,0.44400,0.536,202488,...,0,0,0,0,0,0,0,0,Stage 1,0.708122
2841,1KBxrjLmHQf993Q0cjYbHj,So I,SUPER JUNIOR,2005,31,https://i.scdn.co/image/ab67616d0000b27310a4c7...,Super Junior 05,0.22300,0.548,223213,...,0,0,0,0,0,0,0,0,Stage 1,0.698890
1696,0XwvxY5yMRI7vh55eBRe3B,Butterfly (Prologue Mix),BTS,2016,58,https://i.scdn.co/image/ab67616d0000b273c6dbc6...,The Most Beautiful Moment in Life: Young Forever,0.04460,0.373,295128,...,0,0,0,0,0,0,0,0,Stage 1,0.694253
1721,4C5gHVYvSSJRPsunMJqdsd,Butterfly - Japanese Ver.,BTS,2016,26,https://i.scdn.co/image/ab67616d0000b273a3533c...,Run (Japanese Ver./Standard Edition),0.02180,0.289,241867,...,0,0,0,0,0,0,0,0,Stage 1,0.679788
2778,4ojZUY8fA7RqGo31UdXTuQ,잠들고 싶어 In My Dream,SUPER JUNIOR,2010,20,https://i.scdn.co/image/ab67616d0000b2735e62a2...,미인아 (Bonamana) - The 4th Album Repackage,0.55000,0.280,303787,...,0,0,0,0,0,0,0,0,Stage 1,0.666670
2471,6gGpF8V8miC4O0GZUqSZ9a,Intro : The Invitation,ENHYPEN,2021,50,https://i.scdn.co/image/ab67616d0000b273714e56...,BORDER : CARNIVAL,0.00442,0.283,130883,...,0,0,0,0,0,0,0,0,Stage 1,0.659102
5727,32DeerIMaoGPYUfopOOL40,ADORE YOU,iKON,2019,37,https://i.scdn.co/image/ab67616d0000b2738b9479...,NEW KIDS REPACKAGE : THE NEW KIDS,0.21400,0.500,210575,...,0,0,0,0,0,0,0,0,Stage 1,0.636579
1985,4p3V4c9u09KYZBrzL9dvBG,Eagle Bear Buffalo,Passenger,2018,20,https://i.scdn.co/image/ab67616d0000b273bdd757...,Runaway (Deluxe),0.01190,0.442,204253,...,0,0,0,0,0,0,0,0,Stage 1,0.616225
9394,5kWbITj1O9l3hCWeUPwzJy,Chelsea's Hip Flask,Daniel Pemberton,2022,1,https://i.scdn.co/image/ab67616d0000b273094932...,The Afterparty: Season 1 (Apple TV+ Original S...,0.30000,0.123,149567,...,0,0,0,0,0,0,0,0,Stage 1,0.612390


In [174]:
top_recommendations_2

,track_id,track_name,artist_name,year,popularity,artwork_url,album_name,acousticness,danceability,duration_ms,...,vaporwave,variété française,visual kei,vocal jazz,vocaloid,west coast hip hop,worship,iso_stage,similarity,similarity_2
251,0dcIgqPCMOcCxAL1AuUSpH,Love Somebody,Maroon 5,2023,4,https://i.scdn.co/image/ab67616d0000b2730fea8a...,Workout 2024,0.12900,0.675,229827,...,0,0,0,0,0,0,0,Stage 7,0.509933,0.862770
796,2idIU5vV7T0HO6mspNy0jC,Love Somebody,Maroon 5,2022,2,https://i.scdn.co/image/ab67616d0000b27375dc1b...,Throwback Summer Hits,0.12900,0.675,229827,...,0,0,0,0,0,0,0,Stage 7,0.504656,0.860487
241,3Y0jUr5wCR4AynGb8328uL,Love Somebody,Maroon 5,2023,2,https://i.scdn.co/image/ab67616d0000b2737da160...,2013 Best of by uDiscover,0.12900,0.675,229827,...,0,0,0,0,0,0,0,Stage 7,0.504656,0.860487
8980,6I4jM8zkQVhWBlRWM72TZv,Future Nostalgia (Joe Goddard Remix) [Mixed],"Dua Lipa, The Blessed Madonna, Joe Goddard",2020,34,https://i.scdn.co/image/ab67616d0000b27357fe88...,Club Future Nostalgia (DJ Mix),0.03010,0.774,174240,...,0,0,0,0,0,0,0,Stage 7,0.606832,0.858054
353,1K5tlcaA7exEGO628WK6MO,Love Somebody,Maroon 5,2023,0,https://i.scdn.co/image/ab67616d0000b27368589c...,Para cuando te sientes sin confianza,0.12900,0.675,229827,...,0,0,0,0,0,0,0,Stage 7,0.499278,0.858035
122,5GtBDuWv8TNIGXWTd7o8zR,Sugar,Maroon 5,2023,12,https://i.scdn.co/image/ab67616d0000b273227d18...,Dinner With Friends 2023,0.07470,0.729,234000,...,0,0,0,0,0,0,0,Stage 7,0.567917,0.839374
83,6P8CNYaBLSlDR2rAK1CtmN,Sugar,Maroon 5,2023,11,https://i.scdn.co/image/ab67616d0000b273d200df...,新年あけましておめでとう!２０２４,0.07470,0.729,234000,...,0,0,0,0,0,0,0,Stage 7,0.565682,0.838605
426,1qrWaGOBjWjCzWQF7m1pZT,Lovesick,Maroon 5,2023,1,https://i.scdn.co/image/ab67616d0000b273f87891...,Mix para el orden,0.00248,0.675,185068,...,0,0,0,0,0,0,0,Stage 7,0.476679,0.837742
781,6Im2YIPDPOUU4Kkilow2TM,Sugar,Maroon 5,2022,9,https://i.scdn.co/image/ab67616d0000b27375dc1b...,Throwback Summer Hits,0.07470,0.729,234000,...,0,0,0,0,0,0,0,Stage 7,0.561134,0.836960
163,2JbFx65ckk9QzLdSsri6Ig,Sugar,Maroon 5,2023,8,https://i.scdn.co/image/ab67616d0000b27362e7b8...,Nachtje door,0.07470,0.729,234000,...,0,0,0,0,0,0,0,Stage 7,0.558823,0.836084
